# SMILES → Tanimoto score computed on all fingerprints used in MolForge
Calcula el Tanimoto coefficient entre dos columnes de **SMILES** fent servir les 15 **fingerprints** utilitzades al paper de MolForge

**Entrada**: CSV amb columnes `SMILES_input` i `SMILES_output_ECFP4`.

**Sortida**: CSV amb columnes `MACCS`, `Avalon`, `RDK4`, `RDK4-L`, `HashAP`, `TT`, `HashTT`, `ECFP0`, `ECFP2`, `ECFP4`, `FCFP2`, `FCFP4`, `AEs`, `ECFP2*`, `ECFP4*`.

## Imports

In [1]:
# Per definir l'arrel del projecte
import os

# Pandas pels dataframes
import pandas as pd
import numpy as np # pels NaN

# Per no mostrar els Warnings de RDKit
from rdkit import RDLogger
from rdkit import DataStructs
RDLogger.DisableLog("rdApp.*")

## Inputs (part a editar)

Arrel del projecte

In [2]:
os.chdir("/export/home/ddiestre/MolForge_Testing")

In [3]:
# Importem la funció smiles_to_fingerprint del nostre sourcecode que utilitza RDKit
from src.smiles_to_fp import smiles_to_fingerprint, get_supported_fingerprints
from src.fingerprints import FpSimilarity

Fingerprints en que transformar els SMILES

In [4]:
fp_type = "ECFP4"

Fitxer de fingerprints preprocessat (path a partir de MolForge_Testing/)

In [5]:
input_path = "data/MolForge_output/MolForge_MFoutput_2000_ECFP4.csv"
#input_path = "data/MolForge_output/CoCoGraph_MFoutput_2000_novel.csv"
#input_path = "data/MolForge_output/CoCoGraph_MFoutput_2000_lt70atoms.csv"

SMILES_in_col_name = "SMILES_input"
SMILES_out_col_name = "SMILES_output_" + fp_type

Fitxer en que guardar l'output (path a partir de MolForge_Testing/)

In [6]:
# Output_path ha de ser una carpeta
output_path = "data/analysis_output/MolForge_MF_Analysis_2000_ECFP4/"
#output_path = "data/analysis_output/CoCoGraph_MF_Analysis_2000_novel/"
#output_path = "data/analysis_output/CoCoGraph_MF_Analysis_2000_lt70atoms/"

# Creem la carpeta de output_path
os.makedirs(output_path, exist_ok=True)

# Columna on guardarem l'output de MolForge en format fp_type (per visualitzar els resultats en format bitvectors)
fp_out_col_name = "fingerprints_output_" + fp_type

## 1. Lectura del fitxer

In [7]:
# Lectura del fitxer
df = pd.read_csv(input_path, sep = ',', index_col = 0)
df.head(5)

,SMILES_input,fingerprints_input_ECFP4,SMILES_output_ECFP4
id,,,
1,C C 1 = C ( C = C C ( = C 1 ) C ( C ) ( C ) C ...,1 13 33 80 94 114 118 227 283 351 392 404 444 ...,CC1=C(C=CC(=C1)C(C)(C)C)OC[C@H](C[NH2+]CCCOC)O
2,C ( C ( F ) ( F ) F ) ( C ( F ) ( F ) [18F] ) F,1 114 136 561 1050 1434 1453 1928 2039,C(C(C(C(C(F)(F)F)F)(F)F)(F)F)(C(C(C(F)(F)F)F)(...
3,C C 1 = C C = C C = C 1 C ( C ( = O ) N C 2 C ...,1 14 80 145 196 237 241 325 330 383 387 428 57...,CC1=CC=CC=C1C(C(=O)NC2CCCC2)N(C3=CC=C(C=C3)C(=...
4,C C O C ( = O ) C 1 = C C = C ( C = C 1 ) N C ...,5 41 63 80 89 121 145 147 162 191 216 219 233 ...,CCOC(=O)C1=CC=C(C=C1)N2C(=O)C3C4CC(C3C2=O)C5C4...
5,C 1 [C@H] ( C N ( C 1 = O ) C C 2 = C C = C C ...,80 93 101 172 197 221 231 295 311 314 366 378 ...,C1C(CN(C1=O)CC2=CC=CC=C2Cl)C(=O)NCCC3=CC=CC=N3


## 2. Columna SMILES_output → Columna de fingerprints

Aquesta part del codi ens serveix únicament per visualitzar els resultats de MolForge en forma de fingerprints i per identificar les al·lucinacions.

In [8]:
fingerprints = []

# Apliquem el conversor SMILES -> fingerprint
fingerprints = df[SMILES_out_col_name].apply(
    lambda s: smiles_to_fingerprint(s, fp_type=fp_type, n_bits=2048, return_bits=True)
)

# Guardem les fingerprints en el mateix format que l'input de MolForge
df[fp_out_col_name] = fingerprints.apply(
    lambda lst: " ".join(str(x) for x in lst) if isinstance(lst, list) else lst
)

# Visusalitzem el nou dataframe
df.head(5)

,SMILES_input,fingerprints_input_ECFP4,SMILES_output_ECFP4,fingerprints_output_ECFP4
id,,,,
1,C C 1 = C ( C = C C ( = C 1 ) C ( C ) ( C ) C ...,1 13 33 80 94 114 118 227 283 351 392 404 444 ...,CC1=C(C=CC(=C1)C(C)(C)C)OC[C@H](C[NH2+]CCCOC)O,1 13 33 80 94 114 118 227 283 351 392 404 444 ...
2,C ( C ( F ) ( F ) F ) ( C ( F ) ( F ) [18F] ) F,1 114 136 561 1050 1434 1453 1928 2039,C(C(C(C(C(F)(F)F)F)(F)F)(F)F)(C(C(C(F)(F)F)F)(...,1 114 136 382 561 565 569 588 1050 1205 1434 1...
3,C C 1 = C C = C C = C 1 C ( C ( = O ) N C 2 C ...,1 14 80 145 196 237 241 325 330 383 387 428 57...,CC1=CC=CC=C1C(C(=O)NC2CCCC2)N(C3=CC=C(C=C3)C(=...,1 14 80 145 196 237 241 325 330 383 387 428 57...
4,C C O C ( = O ) C 1 = C C = C ( C = C 1 ) N C ...,5 41 63 80 89 121 145 147 162 191 216 219 233 ...,CCOC(=O)C1=CC=C(C=C1)N2C(=O)C3C4CC(C3C2=O)C5C4...,5 41 63 80 89 121 145 147 162 191 216 219 233 ...
5,C 1 [C@H] ( C N ( C 1 = O ) C C 2 = C C = C C ...,80 93 101 172 197 221 231 295 311 314 366 378 ...,C1C(CN(C1=O)CC2=CC=CC=C2Cl)C(=O)NCCC3=CC=CC=N3,80 93 101 172 197 221 231 295 311 314 366 378 ...


## 3. Càlcul del Tanimoto coefficient entre cada parell de SMILES per tots els fingerprints

In [9]:
for col in get_supported_fingerprints():
    
    fingerprints_input = []
    # Apliquem el conversor SMILES -> fingerprint
    fingerprints_input = df[SMILES_in_col_name].apply(
        lambda s: smiles_to_fingerprint(s, fp_type=col, n_bits=2048, return_bits=False)
    )

    fingerprints_output = []
    # Apliquem el conversor SMILES -> fingerprint
    fingerprints_output = df[SMILES_out_col_name].apply(
        lambda s: smiles_to_fingerprint(s, fp_type=col, n_bits=2048, return_bits=False)
    )

    # Considerem un SMILES Invàlid com que té Tanimoto 0
    tanimotos_Inv_0 = [
        DataStructs.TanimotoSimilarity(fp_in, fp_out)*100 # multipliquem per 100 per tenir-ho en percentatge
        # guardem 0 en els casos on l'output no és correcte i NaN si l'input és NaN
        if (not pd.isna(fp_out) and fp_out != "InvalidSMILE")
        else (0 if fp_out == "InvalidSMILE" else np.nan)
        for fp_in, fp_out in zip(fingerprints_input, fingerprints_output)
    ]

    # No Considerem els SMILES Invàlids
    tanimotos_Inv_NaN = [
        DataStructs.TanimotoSimilarity(fp_in, fp_out)*100 # multipliquem per 100 per tenir-ho en percentatge
        # guardem NaN en els casos on l'output no és correcte i NaN si l'input és NaN
        if (not pd.isna(fp_out) and fp_out != "InvalidSMILE")
        else (np.nan if fp_out == "InvalidSMILE" else np.nan)
        for fp_in, fp_out in zip(fingerprints_input, fingerprints_output)
    ]

    df[col+"_Inv_0"] = tanimotos_Inv_0
    df[col+"_Inv_NaN"] = tanimotos_Inv_NaN

Guardem el dataframe df complet amb els Tanimotos de cada parell de SMILES

In [10]:
df.to_csv(output_path+"Individual_Tanimotos.csv")
df.head(5)

,SMILES_input,fingerprints_input_ECFP4,SMILES_output_ECFP4,fingerprints_output_ECFP4,MACCS_Inv_0,MACCS_Inv_NaN,Avalon_Inv_0,Avalon_Inv_NaN,RDK4_Inv_0,RDK4_Inv_NaN,...,FCFP2_Inv_0,FCFP2_Inv_NaN,FCFP4_Inv_0,FCFP4_Inv_NaN,AEs_Inv_0,AEs_Inv_NaN,ECFP2*_Inv_0,ECFP2*_Inv_NaN,ECFP4*_Inv_0,ECFP4*_Inv_NaN
id,,,,,,,,,,,,,,,,,,,,,
1,C C 1 = C ( C = C C ( = C 1 ) C ( C ) ( C ) C ...,1 13 33 80 94 114 118 227 283 351 392 404 444 ...,CC1=C(C=CC(=C1)C(C)(C)C)OC[C@H](C[NH2+]CCCOC)O,1 13 33 80 94 114 118 227 283 351 392 404 444 ...,100.0,100.0,100.0,100.0,100.000000,100.000000,...,100.000000,100.000000,100.00000,100.00000,100.000000,100.000000,100.0,100.0,100.000000,100.000000
2,C ( C ( F ) ( F ) F ) ( C ( F ) ( F ) [18F] ) F,1 114 136 561 1050 1434 1453 1928 2039,C(C(C(C(C(F)(F)F)F)(F)F)(F)F)(C(C(C(F)(F)F)F)(...,1 114 136 382 561 565 569 588 1050 1205 1434 1...,100.0,100.0,28.0,28.0,61.904762,61.904762,...,32.258065,32.258065,29.72973,29.72973,32.258065,32.258065,70.0,70.0,44.444444,44.444444
3,C C 1 = C C = C C = C 1 C ( C ( = O ) N C 2 C ...,1 14 80 145 196 237 241 325 330 383 387 428 57...,CC1=CC=CC=C1C(C(=O)NC2CCCC2)N(C3=CC=C(C=C3)C(=...,1 14 80 145 196 237 241 325 330 383 387 428 57...,100.0,100.0,100.0,100.0,100.000000,100.000000,...,100.000000,100.000000,100.00000,100.00000,100.000000,100.000000,100.0,100.0,100.000000,100.000000
4,C C O C ( = O ) C 1 = C C = C ( C = C 1 ) N C ...,5 41 63 80 89 121 145 147 162 191 216 219 233 ...,CCOC(=O)C1=CC=C(C=C1)N2C(=O)C3C4CC(C3C2=O)C5C4...,5 41 63 80 89 121 145 147 162 191 216 219 233 ...,100.0,100.0,100.0,100.0,100.000000,100.000000,...,100.000000,100.000000,100.00000,100.00000,100.000000,100.000000,100.0,100.0,100.000000,100.000000
5,C 1 [C@H] ( C N ( C 1 = O ) C C 2 = C C = C C ...,80 93 101 172 197 221 231 295 311 314 366 378 ...,C1C(CN(C1=O)CC2=CC=CC=C2Cl)C(=O)NCCC3=CC=CC=N3,80 93 101 172 197 221 231 295 311 314 366 378 ...,100.0,100.0,100.0,100.0,100.000000,100.000000,...,100.000000,100.000000,100.00000,100.00000,100.000000,100.000000,100.0,100.0,100.000000,100.000000


## 4. Tanimoto promig per Fingerprint emprada per calcular-lo (columnes)

In [11]:
avg_tanimotos_Inv_0 = {col: df[col+"_Inv_0"].mean() for col in get_supported_fingerprints()}
Tanimoto_Inv_0_df = pd.DataFrame([avg_tanimotos_Inv_0], index=[fp_type])

Tanimoto_Inv_0_df

,MACCS,Avalon,RDK4,RDK4-L,HashAP,TT,HashTT,ECFP0,ECFP2,ECFP4,FCFP2,FCFP4,AEs,ECFP2*,ECFP4*
ECFP4,98.46554,97.243693,97.831345,97.982624,94.162854,94.835785,94.841257,96.111084,95.790468,95.07094,95.956494,95.391725,95.785758,98.642006,97.165778


In [12]:
avg_tanimotos_Inv_NaN = {col: df[col+"_Inv_NaN"].mean() for col in get_supported_fingerprints()}
Tanimoto_Inv_NaN_df = pd.DataFrame([avg_tanimotos_Inv_NaN], index=[fp_type])

Tanimoto_Inv_NaN_df

,MACCS,Avalon,RDK4,RDK4-L,HashAP,TT,HashTT,ECFP0,ECFP2,ECFP4,FCFP2,FCFP4,AEs,ECFP2*,ECFP4*
ECFP4,98.662866,97.43857,98.0274,98.178982,94.351558,95.025836,95.031319,96.303691,95.982433,95.261463,96.148792,95.58289,95.977714,98.839685,97.360499


## 5. Càlcul d'altres paràmetres interessants

### Average tanimoto score

In [13]:
Tanimoto_Inv_0_df["Avg_Tc"] = Tanimoto_Inv_0_df.mean(axis=1)
Tanimoto_Inv_NaN_df["Avg_Tc"] = Tanimoto_Inv_NaN_df.mean(axis=1)

### Percentatge d'al·lucinacions de MolForge

In [14]:
Tanimoto_Inv_0_df["Invalid (%)"] = (df[fp_out_col_name] == "InvalidSMILE").mean() * 100
Tanimoto_Inv_NaN_df["Invalid (%)"] = (df[fp_out_col_name] == "InvalidSMILE").mean() * 100

### Percentatge de string-exacts

In [15]:
Tanimoto_Inv_0_df["String-exacts (%)"] = (Tanimoto_Inv_0_df["Avg_Tc"] == 100).mean() * 100
Tanimoto_Inv_NaN_df["String-exacts (%)"] = (Tanimoto_Inv_NaN_df["Avg_Tc"] == 100).mean() * 100

## 6. Guardar el dataframe dels resultats de l'anàlisi de la Tanimoto score

In [16]:
Tanimoto_Inv_0_df.to_csv(output_path+"Tanimoto_Inv_0.csv")
Tanimoto_Inv_0_df

,MACCS,Avalon,RDK4,RDK4-L,HashAP,TT,HashTT,ECFP0,ECFP2,ECFP4,FCFP2,FCFP4,AEs,ECFP2*,ECFP4*,Avg_Tc,Invalid (%),String-exacts (%)
ECFP4,98.46554,97.243693,97.831345,97.982624,94.162854,94.835785,94.841257,96.111084,95.790468,95.07094,95.956494,95.391725,95.785758,98.642006,97.165778,96.351823,0.2,0.0


In [17]:
Tanimoto_Inv_NaN_df.to_csv(output_path+"Tanimoto_Inv_NaN.csv")
Tanimoto_Inv_NaN_df

,MACCS,Avalon,RDK4,RDK4-L,HashAP,TT,HashTT,ECFP0,ECFP2,ECFP4,FCFP2,FCFP4,AEs,ECFP2*,ECFP4*,Avg_Tc,Invalid (%),String-exacts (%)
ECFP4,98.662866,97.43857,98.0274,98.178982,94.351558,95.025836,95.031319,96.303691,95.982433,95.261463,96.148792,95.58289,95.977714,98.839685,97.360499,96.544913,0.2,0.0
